In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 23.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=df18a39aaa567decafc8725a5fbab94dc51fe2241e7c52ff3ac25cc0adcf880e
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark


In [2]:
filepath = "drive/MyDrive/adv_analytics/all_blocks.csv"

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('Adv_Analytics').getOrCreate()

In [6]:
prev = spark.read.csv(filepath)

In [7]:
prev

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string]

In [8]:
prev.show(3)

+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
|  _c0|  _c1|              _c2|         _c3|         _c4|         _c5|    _c6|   _c7|   _c8|   _c9|   _c10|    _c11|
+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|     cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
|37291|53113|0.833333333333333|           ?|         1.0|           ?|      1|     1|     1|     1|      0|    True|
|39086|47614|                1|           ?|         1.0|           ?|      1|     1|     1|     1|      1|    True|
+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
only showing top 3 rows



In [ ]:
# Doing Schema Inference and Missing Values Set to Null -> the column names are set correctly and the ? strings have been replaced by null values

In [9]:
parsed = spark.read.option("header", "true").option("nullValue", "?").\
option("inferSchema", "true").csv(filepath)

In [10]:
parsed.printSchema()

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)
 |-- cmp_fname_c2: double (nullable = true)
 |-- cmp_lname_c1: double (nullable = true)
 |-- cmp_lname_c2: double (nullable = true)
 |-- cmp_sex: integer (nullable = true)
 |-- cmp_bd: integer (nullable = true)
 |-- cmp_bm: integer (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)



In [ ]:
# Schema Inference does two passes -> one pass to figure out the type of each column, and a second pass to do the actual parsing.
# If you know the schema that you want to use for a file ahead of time, you can create an instance of the pyspark.sql.types.StructType 
# class and pass it to the Reader # API via the schema function. This can have a significant performance benefit when the dataset is 
# very large, since Spark will not need to perform an extra pass over the data to figure out the data type of each column.

In [ ]:
#from pyspark.sql.types import *

In [ ]:
#schema = StructType([StructField("id_1", IntegerType(), False),
#StructField("id_2", StringType(), False),
#StructField("cmp_fname_c1", DoubleType(), False)])

In [ ]:
#parsed2 = spark.read.schema(schema).csv(filepath)

In [ ]:
# DataFrames have a number of methods that enable us to read data from the cluster into the PySpark REPL on our client machine.

In [11]:
parsed.first()

Row(id_1=37291, id_2=53113, cmp_fname_c1=0.833333333333333, cmp_fname_c2=None, cmp_lname_c1=1.0, cmp_lname_c2=None, cmp_sex=1, cmp_bd=1, cmp_bm=1, cmp_by=1, cmp_plz=0, is_match=True)

In [ ]:
# If we know the dataset is small -> only then we can use the toPandas or collect method to return all the contents of a DataFrame to the client as an array. 

In [ ]:
# Analyzing Data with the DataFrame API

In [12]:
# getting an idea of the number of records
parsed.count()

5749132

In [13]:
# Whenever we ask another question -> do another computation, Spark will do these same operations, again and again, even if we have filtered the 
# Analyzing Data with the DataFrame API data down to a small number of records or are working with an aggregated version of the original dataset.
# This isn’t an optimal use of our compute resources. After the data has been parsed once, we’d like to save the data in its parsed form on the 
# cluster so that we don’t have to reparse it every time
parsed.cache()

DataFrame[id_1: int, id_2: int, cmp_fname_c1: double, cmp_fname_c2: double, cmp_lname_c1: double, cmp_lname_c2: double, cmp_sex: int, cmp_bd: int, cmp_bm: int, cmp_by: int, cmp_plz: int, is_match: boolean]

In [14]:
# we want to know is the relative fraction of records that were matches versus those that were nonmatches

from pyspark.sql.functions import col
parsed.groupBy("is_match").count().orderBy(col("count").desc()).show()

+--------+-------+
|is_match|  count|
+--------+-------+
|   false|5728201|
|    true|  20931|
+--------+-------+



In [15]:
# In addition to count, we can also compute more complex aggregations like sums, mins, maxes, means, and 
# standard deviation using the agg method of the DataFrame API in conjunction with the aggregation functions 
# defined in the pyspark.sql.functions collection

from pyspark.sql.functions import avg, stddev
parsed.agg(avg("cmp_sex"), stddev("cmp_sex")).show()

+-----------------+--------------------+
|     avg(cmp_sex)|stddev_samp(cmp_sex)|
+-----------------+--------------------+
|0.955001381078048| 0.20730111116897532|
+-----------------+--------------------+



In [ ]:
# we have the option to treat any DataFrame we create as if it were a database 
# table and to express our questions using familiar and powerful SQL syntax.

In [16]:
# we need to tell the Spark SQL execution engine the name it should associate with the parsed DataFrame

parsed.createOrReplaceTempView("linkage")

In [17]:
# Once our temporary table is registered with the Spark SQL engine, we can query it

spark.sql("""
SELECT is_match, COUNT(*) cnt
FROM linkage
GROUP BY is_match
ORDER BY cnt DESC
""").show()

+--------+-------+
|is_match|    cnt|
+--------+-------+
|   false|5728201|
|    true|  20931|
+--------+-------+



In [ ]:
# Fast Summary Statistics for DataFrames

In [18]:
# Computing the min, max, mean, and standard deviation of all
# the non-null values in the numerical columns of a dataframe

summary = parsed.describe()
summary.show()

+-------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|summary|              id_1|              id_2|      cmp_fname_c1|       cmp_fname_c2|       cmp_lname_c1|       cmp_lname_c2|            cmp_sex|             cmp_bd|             cmp_bm|             cmp_by|            cmp_plz|
+-------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|  count|           5749132|           5749132|           5748125|             103698|            5749132|               2464|            5749132|            5748337|            5748337|            5748337|            5736289|
|   mean| 33324.48559643438| 66587.43558331935|0.7129024704419248| 0.9000176718903281| 0.315

In [19]:
# use the select method to choose a subset of the columns 
# to make the summary statistics easier to read and compare:

summary.select("summary", "cmp_fname_c1", "cmp_fname_c2").show()

+-------+------------------+-------------------+
|summary|      cmp_fname_c1|       cmp_fname_c2|
+-------+------------------+-------------------+
|  count|           5748125|             103698|
|   mean|0.7129024704419248| 0.9000176718903281|
| stddev|0.3887583596162795|0.27131761057823317|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



In [ ]:
# Once we have an overall feel for the distribution of the variables in our data, we
# want to understand how the values of those variables are correlated with the value of
# the is_match column.

In [22]:
matches = parsed.where("is_match = true")
match_summary = matches.describe()

In [23]:
misses = parsed.filter(col("is_match") == False)
miss_summary = misses.describe()

In [ ]:
# We can now start to compare our match_summary and miss_summary DataFrames to
# see how the distribution of the variables changes depending on whether the record is
# a match or a miss. 

In [ ]:
# Pivoting and Reshaping DataFrames
# Transposing i.e. Pivot and Reshaping for better analysis


In [24]:
# Use Pandas to do transformation since small dataset
# convert summary into a pandas DataFrame:
summary_p = summary.toPandas()

In [25]:
summary_p.head()


,summary,id_1,id_2,cmp_fname_c1,cmp_fname_c2,cmp_lname_c1,cmp_lname_c2,cmp_sex,cmp_bd,cmp_bm,cmp_by,cmp_plz
0,count,5749132,5749132,5748125,103698,5749132,2464,5749132,5748337,5748337,5748337,5736289
1,mean,33324.48559643438,66587.43558331935,0.7129024704419248,0.9000176718903281,0.3156278193069733,0.31841283153174393,0.955001381078048,0.22446526708507172,0.48885529849763504,0.2227485966810923,0.00552866147434343
2,stddev,23659.859374487736,23620.487613269994,0.3887583596162795,0.27131761057823317,0.33423363396159295,0.3685670662006656,0.20730111116897532,0.4172297223846254,0.49987582367791233,0.41609096298317527,0.07414914925419852
3,min,1,6,0.0,0.0,0.0,0.0,0,0,0,0,0
4,max,99980,100000,1.0,1.0,1.0,1.0,1,1,1,1,1


In [26]:
summary_p.shape

(5, 12)

In [27]:
summary_p = summary_p.set_index('summary').transpose().reset_index()
summary_p = summary_p.rename(columns={'index':'field'})
summary_p = summary_p.rename_axis(None, axis=1)
summary_p.shape

(11, 6)

In [28]:
# Convert it into a Spark DataFrame using SparkSession’s createDataFrame method

summaryT = spark.createDataFrame(summary_p)
summaryT.show()

+------------+-------+-------------------+-------------------+---+------+
|       field|  count|               mean|             stddev|min|   max|
+------------+-------+-------------------+-------------------+---+------+
|        id_1|5749132|  33324.48559643438| 23659.859374487736|  1| 99980|
|        id_2|5749132|  66587.43558331935| 23620.487613269994|  6|100000|
|cmp_fname_c1|5748125| 0.7129024704419248| 0.3887583596162795|0.0|   1.0|
|cmp_fname_c2| 103698| 0.9000176718903281|0.27131761057823317|0.0|   1.0|
|cmp_lname_c1|5749132| 0.3156278193069733|0.33423363396159295|0.0|   1.0|
|cmp_lname_c2|   2464|0.31841283153174393| 0.3685670662006656|0.0|   1.0|
|     cmp_sex|5749132|  0.955001381078048|0.20730111116897532|  0|     1|
|      cmp_bd|5748337|0.22446526708507172| 0.4172297223846254|  0|     1|
|      cmp_bm|5748337|0.48885529849763504|0.49987582367791233|  0|     1|
|      cmp_by|5748337| 0.2227485966810923|0.41609096298317527|  0|     1|
|     cmp_plz|5736289|0.00552866147434

In [29]:
# Print the schema of the summaryT DataFrame
summaryT.printSchema()

root
 |-- field: string (nullable = true)
 |-- count: string (nullable = true)
 |-- mean: string (nullable = true)
 |-- stddev: string (nullable = true)
 |-- min: string (nullable = true)
 |-- max: string (nullable = true)



In [30]:
# Since we want to analyze the summary statistics as numbers, 
# we’ll need to convert the values from strings to double

from pyspark.sql.types import DoubleType
for c in summaryT.columns:
    if c == 'field':
        continue
    summaryT = summaryT.withColumn(c, summaryT[c].cast(DoubleType()))

summaryT.printSchema()

root
 |-- field: string (nullable = true)
 |-- count: double (nullable = true)
 |-- mean: double (nullable = true)
 |-- stddev: double (nullable = true)
 |-- min: double (nullable = true)
 |-- max: double (nullable = true)



In [31]:
#  let’s implement our logic into a function that we can reuse on 
# the match_summary and miss_summary DataFrames:

from pyspark.sql import DataFrame
from pyspark.sql.types import DoubleType
def pivot_summary(desc):
    # convert to pandas dataframe
    desc_p = desc.toPandas()
    # transpose
    desc_p = desc_p.set_index('summary').transpose().reset_index()
    desc_p = desc_p.rename(columns={'index':'field'})
    desc_p = desc_p.rename_axis(None, axis=1)
    # convert to Spark dataframe
    descT = spark.createDataFrame(desc_p)
    # convert metric columns to double from string
    for c in descT.columns:
        if c == 'field':
            continue
        else:
            descT = descT.withColumn(c, descT[c].cast(DoubleType()))
    return descT


match_summaryT = pivot_summary(match_summary)
miss_summaryT = pivot_summary(miss_summary)

In [ ]:
# Joining DataFrames and Selecting Features
# Although the DataFrame API includes a join function, it’s often easier to express 
# these joins using Spark SQL, especially when the tables we are joining have a 
# large number of column names in common and we want to be able to clearly 
# indicate which column we are referring to in our select expressions


In [32]:
# Create temporary views for the match_summaryT and miss_summaryT DataFrames, 
# join them on the field column, and compute some simple summary 
# statistics on the resulting rows:

match_summaryT.createOrReplaceTempView("match_desc")
miss_summaryT.createOrReplaceTempView("miss_desc")

spark.sql("""
SELECT a.field, a.count + b.count total, a.mean - b.mean delta
FROM match_desc a INNER JOIN miss_desc b ON a.field = b.field
WHERE a.field NOT IN ("id_1", "id_2")
ORDER BY delta DESC, total DESC
""").show()


+------------+---------+--------------------+
|       field|    total|               delta|
+------------+---------+--------------------+
|     cmp_plz|5736289.0|  0.9563812499852176|
|cmp_lname_c2|   2464.0|  0.8064147192926266|
|      cmp_by|5748337.0|  0.7762059675300512|
|      cmp_bd|5748337.0|   0.775442311783404|
|cmp_lname_c1|5749132.0|   0.683877248260476|
|      cmp_bm|5748337.0|  0.5109496938298685|
|cmp_fname_c1|5748125.0|  0.2854529057477815|
|cmp_fname_c2| 103698.0|  0.0910426806227922|
|     cmp_sex|5749132.0|0.032408185250332844|
+------------+---------+--------------------+



In [ ]:
# ANALYSIS
# A good feature has two properties: it tends to have significantly different values for
# matches and nonmatches (so the difference between the means will be large), and it
# occurs often enough in the data that we can rely on it to be regularly available for any
# pair of records. By this measure, cmp_fname_c2 isn’t very useful because it’s missing
# a lot of the time, and the difference in the mean value for matches and nonmatches
# is relatively small—0.09, for a score that ranges from 0 to 1.

# The cmp_sex feature also isn’t particularly helpful because even though it’s available 
# for any pair of records, the difference in means is just 0.03.

# Features cmp_plz and cmp_by, on the other hand, are excellent. They almost always
# occur for any pair of records, and there is a very large difference in the mean values
# (more than 0.77 for both features). Features cmp_bd, cmp_lname_c1, and cmp_bm also
# seem beneficial: they are generally available in the dataset, and the difference in mean
# values for matches and nonmatches is substantial

# Features cmp_fname_c1 and cmp_lname_c2 are more of a mixed bag: cmp_fname_c1
# doesn’t discriminate all that well (the difference in the means is only 0.28) even
# though it’s usually available for a pair of records, whereas cmp_lname_c2 has a large
# difference in the means, but it’s almost always missing. It’s not quite obvious under
# what circumstances we should include these features in our model based on this data.

In [33]:
# Scoring and Model Evaluation
# For our scoring function, we are going to sum up the value of five fields
# (cmp_lname_c1, cmp_plz, cmp_by, cmp_bd, and cmp_bm).

# Create the required expression string
good_features = ["cmp_lname_c1", "cmp_plz", "cmp_by", "cmp_bd", "cmp_bm"]
sum_expression = " + ".join(good_features)
sum_expression

'cmp_lname_c1 + cmp_plz + cmp_by + cmp_bd + cmp_bm'

In [34]:
# Use the sum_expression string for calculating the score. When summing up the values, 
# we will account for and replace null values with 0 using DataFrame’s fillna method

from pyspark.sql.functions import expr
scored = parsed.fillna(0, subset=good_features).\
withColumn('score', expr(sum_expression)).\
select('score', 'is_match')
scored.show()


+-----+--------+
|score|is_match|
+-----+--------+
|  4.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  4.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  4.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
+-----+--------+
only showing top 20 rows



In [ ]:
# The final step in creating our scoring function is to decide what threshold 
# the score must exceed in order for us to predict that the two records 
#represent a match. 

In [35]:
# To help us choose a threshold, it’s helpful to create a contingency table (which is
# sometimes called a cross tabulation, or crosstab) that counts the number of records
# whose scores fall above/below the threshold value crossed with the number of records
# in each of those categories that were/were not matches

def crossTabs(scored: DataFrame, t: DoubleType) -> DataFrame:
    return scored.selectExpr(f"score >= {t} as above", "is_match").\
        groupBy("above").pivot("is_match", ("true", "false")).\
        count()

In [36]:
crossTabs(scored, 2.0).show()

+-----+-----+-------+
|above| true|  false|
+-----+-----+-------+
| true|20931| 596414|
|false| null|5131787|
+-----+-----+-------+



In [37]:
crossTabs(scored, 3.0).show()

+-----+-----+-------+
|above| true|  false|
+-----+-----+-------+
| true|20916| 315213|
|false|   15|5412988|
+-----+-----+-------+



In [38]:
crossTabs(scored, 4.0).show()

+-----+-----+-------+
|above| true|  false|
+-----+-----+-------+
| true|20871|    637|
|false|   60|5727564|
+-----+-----+-------+



In [39]:
crossTabs(scored, 5.0).show()

+-----+-----+-------+
|above| true|  false|
+-----+-----+-------+
| true|19697|      5|
|false| 1234|5728196|
+-----+-----+-------+



In [ ]:
# By applying a high threshold value of 4.0, meaning that the average of the five
# features is 0.8, we can filter out almost all of the nonmatches while keeping 
# over 90% of the matches
